# QCoDeS Example with Keysight 344xxA

`344xxA` models of Keysight digital multimeters have similar QCoDeS drivers. In this tutorial, `Keysight_34465A` is chosen for showcasing the usage of the instrument.

Note however that not every feature/parameter is available on all `344xxA` models. This, when possible, is reflected in the instantiated driver object. Also note that models like `34465A` have options (like `DIG` and `MEM`) that can either be enabled or disabled on a particular instrument; this also has impact on availability of some features/parameters and/or their settings. In general, refer to the instrument's manual for detailed information.

__NOTE__: Beginning with firmware revision 3.0, the digitizing and advanced triggering option, referred to as "DIG", for models 34465A-DIG/34470A-DIG, is now standard.

The driver does not cover all the features of the instrument because it is being mostly used for voltage DC measurements. Contribution is welcome.

__NOTE__: use drivers for these models which originate from `*_submodules` python modules, from example:
```python
from qcodes.instrument_drivers.Keysight.Keysight_34465A_submodules import Keysight_34465A
```
instead of
```python
from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
```
The latter ones are deprecated due to their deficiencies.

In [1]:
import time

import numpy

import qcodes
from qcodes.instrument_drivers.Keysight.Keysight_34465A_submodules import Keysight_34465A

In [2]:
dmm = Keysight_34465A('dmm', 'USB0::10893::257::MY57504787::0::INSTR')

Connected to: Keysight Technologies 34465A (serial:MY57504787, firmware:A.02.16-02.40-02.16-00.51-03-01) in 0.46s


In [3]:
dmm.reset()

## Parameters and methods overview

Here is an overview (not exhaustive) of the parameters and methods that are available in the driver.

The driver is not only comprised of root-level parameters and methods but also contains submodules which logically group some functionality.

* Measurements
  * `dmm.init_measurement()`
  * `dmm.fetch()`
  * `dmm.read()`
  * `dmm.volt` - measure one voltage point now (note: may not always work)
* Range settings
  * `dmm.range`
  * `dmm.autorange`
  * `dmm.autorange_once()`
* Triggering
  * `dmm.trigger.source`
  * `dmm.trigger.delay`
  * `dmm.trigger.auto_delay_enabled`
  * `dmm.trigger.count`
  * `dmm.trigger.slope`
  * `dmm.trigger.level`
  * `dmm.trigger.force()`
* Sample settings
  * `dmm.sample.count`
  * `dmm.sample.source`
  * `dmm.sample.timer`
  * `dmm.sample.pretrigger_count`
* Display control
  * `dmm.display.text`
  * `dmm.display.clear()`
  * `dmm.display.enabled`
* Related measurement accuracy
  * `dmm.NPLC`
  * `dmm.resolution`
  * `dmm.line_frequency`
  * `dmm.aperture.mode`
  * `dmm.aperture.time`
  * `dmm.autozero`
  
Use `dmm.parameters` (or other ways) to explore all the parameters.

As an extra, let's print the readable snapshot of the instrument.

In [4]:
dmm.print_readable_snapshot(update=True)

dmm:
	parameter     value
--------------------------------------------------------------------------------
IDN            :	{'vendor': 'Keysight Technologies', 'model': '34465A', 'seri...
NPLC           :	10 (NPLC)
aperture_mode  :	OFF 
aperture_time  :	0.1 
autorange      :	ON 
autozero       :	ON 
line_frequency :	50 (Hz)
range          :	1000 
resolution     :	0.0001 (V)
timeout        :	5 (s)
volt           :	-5.9611e-08 (V)
dmm_display:
	parameter value
--------------------------------------------------------------------------------
enabled :	True 
text    :	 
dmm_trigger:
	parameter         value
--------------------------------------------------------------------------------
auto_delay_enabled :	True 
count              :	1 
delay              :	0.00016 (s)
level              :	0 (V)
slope              :	NEG 
source             :	IMM 
dmm_sample:
	parameter       value
--------------------------------------------------------------------------------
count            :	1 
pretrigg

## Single value reading

If one simpy wants to measure a single voltage value right now, the convenient `volt` parameter can be used.

Note that it may not work for some configurations of the instrument, for example, when `sample.count` is not `1`. Later in this notebook, we will present more prowerful and robust ways of performing measurements.

In [5]:
dmm.volt()

-9.30337575e-06

## Multivalue triggered measurements

__NOTE__: Refer to the instrument manual for more information on how to perform measurements with the instrument; here, only the most basic and frequently used ones are demonstated.

Measurements with the instrument are performed in the following way: the instrument's settings are set for a particular kind of measurement, then the measurement is started/initialized, then after all the data has been acquired, it is retrieved from the instrument. Below is an example of such a measurement.

Use `range`, `autorange` parameters or `autorange_once` method to set the measurement range. Disabling autorange is recommended by the instrument's manual for speeding up the measurement.

In [6]:
dmm.autorange_once()

In order to set up the accuracy of the measurements and related settings, set up `NPLC` or `aperture_*` parameters (if available).

In [7]:
dmm.aperture_mode('ON')
dmm.aperture_time(2e-5)

Set up triggering mechanism. Note that trigger settings and methods are inside `trigger` submodule of the instrument driver. Here, we will use immediate triggering (the measurement is triggered uppon measurement initialization, that is when `init_measurement` is called) with 1 trigger without any delays. Consulm the instrument's manual for more information on various triggering options.

In [8]:
dmm.trigger.source('IMM')
dmm.trigger.count(1)
dmm.trigger.delay(0.0)

Set up sampling settings. Note that sampling parameters and method are inside `sample` submodule of the instrument driver. Here, we set to measure 15 samples.

In [9]:
dmm.sample.count(15)
dmm.sample.pretrigger_count(0)

We are also going to set the sample source to timer (not avaliable in all models) so that the instrument ensures that the samples are taken with fixed periods between them. The `timer` parameter allows to set the value of that fixed period. For simplicity, we are going to let the instrument deduct the minimum value of it according to the current instrument configuration (!) and set it.

In [10]:
dmm.sample.source('TIM')
dmm.sample.timer('MIN')

It turns out that commands are executed faster when the display of the instrument is disabled or is displaying text. One of the further section expands on it. Here, we will just set the display to some text.

In [11]:
dmm.display.text('Example with 15 samples')

In order to initiate the measurement, call `init_measurement` method of the driver. In the case of this example, the instrument will get into "wait-for-trigger" mode but because the trigger source is "immediate", the instrument will immediately start measuring the 15 samples.

In [12]:
dmm.init_measurement()

The insrtument is going to measure 15 samples and save them to its memory. Once the measurement is completed, we can call `fetch` method of the driver to retreive the acquired data.

While the measurement is going, there are two things we can do. One is to `sleep` until the end of the measurement, and then call `fetch`. The other one is to call `fetch` immediately after the measurement has been initiated - this way the instrument will return the acquired data right when the measurement is finished. This sounds pretty useful, however there are two considerations to keep in mind: the instrument manual hints that calling fetching immediately after initiation may be slower than waiting for the measurement to complete; if the measurement takes longer than the VISA command timeout, the code may raise a VISA timeout exception while the measurement is properly running. To overcome the latter consideration, one could in principle wrap `fetch` call into the following `try-finally` block that takes care of the VISA timeout value:
```python
old_timeout = dmm.visa_handle.timeout  # it is in milliseconds
new_timeout = old_timeout + n_samples * time_per_sample * 1000
# where, n_samples == dmm.sample.count(),
# and time_per_sample == dmm.sample.timer()
dmm.visa_handle.timeout = new_timeout
try:
    data = dmm.fetch()
finally:
    dmm.visa_handle.timeout = old_timeout
```

Assuming that we've just slept or waited enough for the measurement to complete, let's `fetch` the data from the instrument. Note that due to the nature of the `fetch` command of the instrument, one can fetch the same measured data more than once (until, for example, a new measurement has been initiated; refer to the instrument's manual for more information on this).

In [13]:
data = dmm.fetch()
data

array([-0.00469509, -0.00495776, -0.00509389, -0.00502464, -0.00457326,
       -0.00368487, -0.00033184,  0.00395255,  0.00707154,  0.00978691,
        0.01218942,  0.01157326,  0.01100726,  0.01077802,  0.0099947 ])

Note that there is also a `read` method. It's difference from `fetch` is that it also initiates the new measurement. Using `read` might be convenient for some cases while `init_measurement` + `fetch` definitely allow for more control.

Since the measurement is finished, let's bring back the display to life.

In [14]:
dmm.display.clear()

If needed, it is straightforward to calculate a vector of times when the acquired data points were measured; for example, like this:

In [15]:
n = dmm.sample.count()
t = dmm.sample.timer()
setpoints = numpy.linspace(0, n*t, n)
setpoints  # in seconds

array([0.     , 0.00063, 0.00126, 0.00189, 0.00252, 0.00315, 0.00378,
       0.00441, 0.00504, 0.00567, 0.0063 , 0.00693, 0.00756, 0.00819,
       0.00882])

## Special values of some parameters

Some parameters can be set to special values like `MIN`/`MAX`/`DEF` which usually mean minimum/maximum/default, respectively. 

In order to obtain the actual value of the parameter that gets set when setting it to one of these special values, just call the get method of the parameter.

In [16]:
# Find out what the maximum value of `sample_timer` can be
dmm.sample.timer('MAX')
dmm.sample.timer()

3600.0

In [17]:
# Find out what the default value of `sample_timer` is
dmm.sample.timer('DEF')
dmm.sample.timer()

1.0

In [18]:
# Find out what the recommended minumum value of `sample_timer` is
dmm.sample.timer('MIN')
dmm.sample.timer()

0.000588

In [19]:
# Alternatively, if available, use a conveniently implemented
# get-only parameter to find out the actual value,
# for example, for MIN value of `sample_timer` there is such
# a convenient get-only parameter:
dmm.sample.timer_minimum()

0.000588

## Display state impacts command execution speed

Although it is indeed just nice to be able to display useful text on the screen of the instrument, it turn out that disabling the display (or making it display does text instead of all the indicators and values) improves command execution speed from the remote interface and provides basic security (quoted from the instrument's manual).

Hence, the driver provides `display` submodule with `text` parameter that displays a given text on the insrturment, and a `clear` method that clears the text from display.

The driver's `display` submodule also provides `enabled` parameter. When it is set to `False`, the state of the display is such that it does not show anything. Note, however, that displaying text is still possible when the `display.enabled` is `False` (when `display.enabled` is `False`, `display.clear` clears the text from the screen but does not enable it).

In [20]:
# Displays the text
dmm.display.text('Hello, World!')

In [21]:
# Returns display to its normal state
dmm.display.clear()

In [22]:
# Note that a call to `display_clear` also updates 
# the value of the `display_text` parameter:
assert dmm.display.text() == ''

In [23]:
# Display can also be cleared by setting 
# `display_text` to an empty string
dmm.display.text('some text')  # Displays some text
time.sleep(0.5)
dmm.display.text('')  # Returns display to its normal state

In [24]:
# Disables the display, which makes it turn black
dmm.display.enabled(False)

In [25]:
# Shows some text on a disabled display
dmm.display.text("i'm disabled but still showing text")

In [26]:
# Enabling display in this state 
# won't change what's being displayed
dmm.display.enabled(True)

In [27]:
# ... but since the display is now enabled,
# clearing the display will not only remove the text
# but also show all the normal display indicators.
dmm.display.clear()

## Error handling

Use the following methods to read the error queue of the instrument. The instrument has an error queue of length up to 20 messages. The queue message retrieval is first-in-first-out.

In [28]:
# Retrieve the first (i.e. oldest) error message 
# in the queue (and thereby remove from the queue)
dmm.error()

(0, 'No error')

In [29]:
# The entire queue can be flushed out
# using `flush_error_queue` method.
# Printing the messages in enabled by default
# and can be disables with the `verbose` kwarg.

# generate a few errors
for _ in range(3):
    dmm.write('produce an error!')

In [30]:
dmm.flush_error_queue(verbose=True)

-113 Undefined header
-113 Undefined header
-113 Undefined header
0 No error


## Using data_buffer for triggered multisample measurement (deprecated)

__NOTE__: This approach is deprecated and only available in the deprecated Keysight 344xxA drivers (those which do not originate from `*_submodules.py` files).

One may use `data_buffer` `ArrayParameter` in order to perform a triggered measurement of a predefined number of samples with a single trigger that has not delay. The following code shows how to do it.

__NOTE__: The reason this approach is deprecated is that the internal implementation of `data_buffer` is quite specific and sets up the instrument into a mode that is not generally useful. This is why it is recommended that the users use driver methods like `read` and `fetch` in order to implement the measurements. It is also up to user to decide to wrap his code into his own measurement function/routine or even a QCoDeS parameter for his own convenience.

In [31]:
from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A as Keysight_34465A_old

In [32]:
dmm_old = Keysight_34465A_old('dmm_old', 'USB0::10893::257::MY57504787::0::INSTR')

c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument_drivers\Keysight\private\Keysight_344xxA.py:143: UserWarning: This driver is old and will be removed from QCoDeS soon. Please use the Keysight_344xxA classes from the files instrument_drivers/Keysight/Keysight_344xxA_submodules instead.
  UserWarning)


Connected to: Keysight Technologies 34465A (serial:MY57504787, firmware:A.02.16-02.40-02.16-00.51-03-01) in 0.51s


In [33]:
dmm_old.reset()

In [34]:
# Set the sample timer for now to default,
# because due to aperture mode usage above
# there might be configuration compatibility 
# errors on the instrument. They are not harmful,
# we do this just for the sake of a clean demo.
dmm_old.sample_timer('DEF')

In [35]:
# Set the trigger mode. For the sake of this example,
# we are going to use "immediate" which will start measuring
# right after `init_measurement` is called (it will be called
# for you by `data_buffer`'s get method')
dmm_old.trigger_source('IMM')

In [36]:
# Set sample count to number of samples that
# should be acquired
dmm_old.sample_count(5)

In [37]:
# One needs to prepare the `data_buffer` parameter
# and the instrument for this measurement, otherwise
# getting `data_buffer` will raise an exception
dmm_old.data_buffer.prepare()

C:\Users\a-miasta\AppData\Local\Continuum\anaconda3\envs\qcodes\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The function "prepare" is deprecated, because the use of this ArrayParameter is limiting as it represents only a single way of performing measurements and performs preparations that are not generally applicable Use "driver methods like `fetch` and `read` (together with other methods and parameters) and custom functions and parameters implemented on top of them" as an alternative.
  after removing the cwd from sys.path.


In [38]:
# Finally perform the measurement,
# and obtain the measured data
# (note the text on the instrument's display
# during the measurement)
data = dmm_old.data_buffer()

In [39]:
# Now, here's the measured data
data

array([-1.22270673e-05,  2.16857420e-05,  1.70717544e-05,  8.30517780e-06,
        1.91480488e-05])

In [40]:
# Here are the values of setpoints,
# in this case its time in seconds when
# each of the points have been measured relative
# to the first point
dmm_old.data_buffer.setpoints

((0.0,
  0.5056200000000001,
  1.0112400000000001,
  1.5168600000000003,
  2.0224800000000003),)

In [41]:
# The shape of the setpoints reflects
# the number of samples
dmm_old.data_buffer.shape

(5,)

In [42]:
# For internal use, the amount of time used for
# measuring every point is saved into the following 
# attribute:
dmm_old.data_buffer.time_per_point

0.404496

In [43]:
# ... which is equal to the value of the
# `sample_timer` parameter:
dmm_old.sample_timer()

0.404496